In [1]:
from models import *

# ACTIVATE GPU USAGE
 
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'        
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(tf.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

2023-05-16 11:45:20.024794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 11:45:20.529196: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/envs/twinnet/lib/
2023-05-16 11:45:20.529266: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No su

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


2023-05-16 11:45:21.568274: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 11:45:21.572610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 11:45:21.572761: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 11:45:21.576419: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 11:45:21.576554: I tensorflow/compiler/xla/stream_executo

# TRAINING PROCESS - 3D MODEL


In [ ]:
# Define paths

src_path_Source = '/media/hmorales/Skynet/MemNuclei2Nuclei/MemNuclei/trainingImages/'
src_path_Target = '/media/hmorales/Skynet/MemNuclei2Nuclei/Nuclei/trainingImages/'
modeltype = '3d'

dataset = generate_datasets(model_type = modeltype,
                            images_folder_source = src_path_Source,
                            images_folder_target = src_path_Target,
                            images_extension= 'tif',
                            patch_shape = (64,64,64),
                            patch_step = 32,
                            threshold = 0.1,
                            percentiles = (40.0, 99.9999),
                            n_samples = 50000,
                            seed_a = 27,
                            seed_b = 57,
                            shuffle_ = False)               

In [ ]:
# plot some patches
Npairs = 6
figWidth=3 
imagesShape = dataset[0].shape    
maxPlanes = imagesShape[3] - 1  
nImages = imagesShape[0]

idxpatch=np.linspace(0, nImages-1, num=Npairs, dtype=np.int16);
idxpatch=np.random.randint(0,nImages,Npairs)
print(imagesShape)

images1 = dataset[0]
images2 = dataset[1]

def show(ax, image):
    ax.imshow(image, cmap='viridis', vmin=-1, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

   
fig = plt.figure(figsize=(2*figWidth,Npairs*figWidth))
axs = fig.subplots(Npairs,2)
for i in range(Npairs):
    if modeltype == '2d':
        show(axs[i,0], images1[idxpatch[i],:,:])
        show(axs[i,1], images2[idxpatch[i],:,:])
    elif modeltype == '3d':
        #show(axs[i,0], np.amax(images1[idxpatch[i],:,:,:],1))
        #show(axs[i,1], np.amax(images2[idxpatch[i],:,:,:],1))
        show(axs[i,0], images1[idxpatch[i],:,:,int(imagesShape[1]/2)])
        show(axs[i,1], images2[idxpatch[i],:,:,int(imagesShape[1]/2)])
    

plt.subplots_adjust(wspace=0, hspace=0)            
plt.show()

#tiff.imwrite('temp1.tif', np.squeeze(images1), metadata={'axes': 'TZYX'})
#tiff.imwrite('temp2.tif', np.squeeze(images2), metadata={'axes': 'TZYX'})


In [ ]:
# Train the model
outdir = ''

training = train_model_3D(dataset = dataset,
                          batch_size = 1, 
                          epochs = 100, 
                          save_freq = 2,
                          output_dir = outdir) 

In [ ]:
# Predict test image
input_image_path='/media/hmorales/Skynet/MemNuclei2Nuclei/MemNuclei/testImages/G2_TL000_Angle90.tif'
output_name = '/media/hmorales/Skynet/MemNuclei2Nuclei/MemNuclei/testImages/Nuclei_G2_TL000_Angle90.tif'
model_path = '/home/hmorales/WorkSpace/Development/CycleGAN3D/StyleTransferMicroscopyImages-main/3DModel/models/g_model_AtoB_3D_019800.h5'
patch_size = 64
overlap_pct = 0.0
percentiles = (40.0, 99.9999)

prediction = predict_3D(input_image_path,  # path to image
                        percentiles, # for image normalization
                        output_name, # output filename. Will be saved in the working directory.
                        model_path, # path to model (.h5 file)
                        patch_size, # size of patch (single value)
                        overlap_pct) # overlap percentage. Default set to 0.125 for 8 pix overlap for patch size 64.)
          

In [5]:
model_path = '/home/hmorales/WorkSpace/Development/CycleGAN3D/StyleTransferMicroscopyImages-main/2DModel/models/g_model_AtoB_2D_114400.h5'

instance_norm = {'InstanceNormalization': InstanceNormalization}
model = load_model(model_path, instance_norm)

# Save weights and architecture
model.save_weights("/home/hmorales/WorkSpace/Development/CycleGAN3D/StyleTransferMicroscopyImages-main/2DModel/models/g_model_AtoB_2D_114400_weights_only.h5")

# Save model config
json_config = model.to_json()
with open('/home/hmorales/WorkSpace/Development/CycleGAN3D/StyleTransferMicroscopyImages-main/2DModel/models/g_model_AtoB_2D_114400_model_config.json', 'w') as json_file:
    json_file.write(json_config)

